# Task 2 – Investigation Queries on DWH

All SQL-statements are in located in `sql/analysis/*.sql`. 
Furthermore, this notebook includes some queries that were not required for the task, but we liked to investigate our data a little further.

In [ ]:
import sys
from pathlib import Path

repo_root = Path("..").resolve()
if repo_root.as_posix() not in sys.path:
    sys.path.insert(0, repo_root.as_posix())

In [2]:
from src.timetable_etl.config import Settings
from src.timetable_etl.db import connect
from src.timetable_etl.sql import load_sql

settings = Settings.from_env()
conn = connect(settings)

## (Not Task Relevant): Random Stop

This query yields a random stop. We used this query as a little test to verify that our pipeline actually produces the intended.

In [3]:
with conn.cursor() as cur:
    cur.execute(load_sql("analysis/random_stop.sql"))
    row = cur.fetchone()

print(row)

('-5934074318845214338-2509191003-9', 8089080, datetime.datetime(2025, 9, 19, 10, 25, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), datetime.datetime(2025, 9, 19, 10, 25, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))), None, None, None, None)


## (Not Task Relevant): Visaluzation of all Stations

This query was helpful for us to visualize all stations and verify that the coordinates were extracted correctly.

In [4]:
import folium

with conn.cursor() as cur:
    cur.execute(load_sql("analysis/stations_in_bbox.sql"))
    rows = cur.fetchall()

BERLIN_BBOX = {"lat_min": 52.3, "lat_max": 52.7, "lon_min": 13.0, "lon_max": 13.85}
rows = [r for r in rows if r[2] is not None and r[3] is not None and (BERLIN_BBOX["lat_min"] <= r[2] <= BERLIN_BBOX["lat_max"]) and (BERLIN_BBOX["lon_min"] <= r[3] <= BERLIN_BBOX["lon_max"])]

if rows:
    lats = [r[2] for r in rows]
    lons = [r[3] for r in rows]
    center = [sum(lats) / len(lats), sum(lons) / len(lons)]
else:
    center = [52.52, 13.405]

m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")
for eva, name, lat, lon in rows[:2000]:
    folium.CircleMarker([lat, lon], radius=4, popup=f"{name} ({eva})").add_to(m)

m

## Task 2.1 Return Station identifier and coordinates (given name)

This query speaks for itself. The only thing that might be worthy to mention is that it is case-insensitive as shown in the cell below. We were not sure if the query also should return the station name again, but we argued that this is not needed.

In [5]:
def get_station_by_name(conn, station_name: str):
    with conn.cursor() as cur:
        cur.execute(load_sql("analysis/get_station_by_name.sql"), {"name": station_name})
        row = cur.fetchone()
    if row is None:
        return None
    eva, lat, lon = row
    return {"eva": eva, "latitude": lat, "longitude": lon}

station_name = "hackescher Markt"
print(get_station_by_name(conn, station_name))

{'eva': 8089017, 'latitude': 52.522622, 'longitude': 13.402364}


## Task 2.2 Return closest Station

This query is also a rather easy query. We would like to note that we had a deeper look into how to calculate distances on earth: https://medium.com/@manishkp220/haversine-formula-find-distance-between-two-points-2561d66c2d79

We found that theoretically the Haversince Distance is the distance metric that is the physically most accurate. However, we decided that the euclidian distance should be a sufficient heuristic.

In [6]:
def nearest_station(conn, lat: float, lon: float):
    with conn.cursor() as cur:
        cur.execute(load_sql("analysis/nearest_station.sql"), {"lat": lat, "lon": lon})
        return cur.fetchone()

lat, lon = 52.5, 13.5
row = nearest_station(conn, lat, lon)
print(row)

(8089006, 'Betriebsbahnhof Berlin-Rummelsburg', 52.493827, 13.497786, 0.0065580275235746855)


## Task 2.3 Amount of Cancelled Stations per Snapshot

This query first defines a time window based on the given snapshot. It then selects the distinct amount of stop_ids from the stops table where either the arrival or the departure was cancelled and either the arrival time or the departure time is in the time window. 

To achieve that, the query left joins our fact table with the time dimension table two times, one for the arrival and one for the departure time. Left joins are important as they keep stop ids which for example do not have an arrival cancellation time id but a departure cancellation time id. The cross join with the time window is technically a cartesion product but as the time window is just one row, the cross join just adds the time window variables to the scope of the query so that a comparison can happen in the where statement.

In [7]:
snap = "25100113"
with conn.cursor() as cur:
    cur.execute(load_sql("analysis/cancelled_stops_in_snapshot.sql"), {"snap": snap})
    row = cur.fetchone()
print(row)

(11,)


## Task 2.4 Average Delay for a given Station

This query is based on the assumptions that we only count positive delays as actual ''delays''. We argue that a train departung earlier than expected is also not in the interest of the customer and thus, should not be used to decrease the average delay. To ensure this assumptions we use ``WHERE t_ct.ts > t_pt.ts``.

Furthermore, we assume that the **departure** delay is meant in this task and thus, we compare these times.

This query is again case-insensitive. 

In [8]:
def avg_delay_for_station(conn, station_name: str):
    with conn.cursor() as cur:
        cur.execute(load_sql("analysis/avg_departure_delay_for_given_station.sql"), {"station_name": station_name})
        return cur.fetchone()

station = "Berlin Hauptbahnhof"
row = avg_delay_for_station(conn, station)

if row is None:
    print("Station name doesnt exist")
else:
    eva, name, n, avg_s = row
    print(f"{eva} | {name} | n={n} | Avg Delay={avg_s/60:.2f} min")


8011160 | Berlin Hauptbahnhof | n=6932 | Avg Delay=14.81 min


## Task 2.4 Average Delay for all Stations

This is a little bonus query that shows the delays of all stations. We were just interested which is the station with the highest delay :)

In [9]:
import pandas as pd
with conn.cursor() as cur:
    cur.execute(load_sql("analysis/avg_departure_delay_for_all_station.sql"))
    rows = cur.fetchall()

df = pd.DataFrame(rows, columns=["eva", "name", "n", "avg_s"])
df["avg_delay_min"] = df["avg_s"] / 60
df = df[["eva", "name", "n", "avg_delay_min"]]
print(df.to_string(index=False))

    eva                               name     n                 avg_delay_min
8010404                     Berlin-Spandau  7121 15.06712540373543041666666667
8011160                Berlin Hauptbahnhof  6932 14.80698211194460473166666667
8010406         Berlin Zoologischer Garten  4161          10.16798846431146359
8011113                    Berlin Südkreuz  3365            9.2000000000000000
8010255                  Berlin Ostbahnhof  4993           8.11275786100540757
8010405                     Berlin-Wannsee  2849 7.760266760266760266666666667
8011102               Berlin Gesundbrunnen  3847           7.07330387314790746
8010403              Berlin-Charlottenburg  2448          6.932598039215686275
8011306             Berlin-Friedrichstraße  3750 6.860266666666666666666666667
8011162                    Berlin Ostkreuz  5630           6.52113676731793961
8011155                     Alexanderplatz  3959 6.345794392523364486666666667
8011118             Berlin Potsdamer Platz  2769 5.5

In [10]:
conn.close()